# Count points in Tacta

In this notebook we process an image, such that we can automatically count the points from the given game.

Here is an overview of what is done

1. **Load and preprocess the image** - Convert to RGB and grayscale
2. **Apply adaptive thresholding** - Use Gaussian blur and adaptive thresholding to detect edges
3. **Find contours** - Detect all contours in the thresholded image
4. **Filter circular contours** - Select circles with diameter 8-18px, white/gray fill, and low color saturation
5. **Extract donut region colors** - Sample HSV colors from rings around each detected circle
6. **Visualize donut regions** - Display circles with inverted colors in their surrounding regions
7. **Define reference colors** - Set up known game colors (Red, Yellow, Green, Blue, Pink) in HSV space
8. **Plot 3D color distribution** - Visualize detected colors and reference colors in 3D cylindrical coordinates
9. **Perform k-means clustering** - Group detected colors into 6 clusters
10. **Map clusters to game colors** - Assign each cluster to the closest reference color for final counting

We mainly use cv2, matplotlib (& plotly for better interactivity for some of the plots) and numpy. See `env.yml`
for an easy to setup environment. Run

```
$ conda env create -f env.yml
```

and use the new environment `tacta` to run this notebook.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

`%matplotlib widget` enables interactive matplotlib plots in Jupyter notebooks, allowing zooming, panning,
and dynamic updates within the notebook interface.

In [ ]:
%matplotlib widget

## Load and preprocess image

In [ ]:
# Load and display the image
image_path = "data/cropped_black.jpg"
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)

In [ ]:
plt.figure(figsize=(8, 10))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("Original Image")
plt.show()

In [ ]:
# Apply adaptive thresholding
blurred = cv2.GaussianBlur(gray, (11, 11), 0)
adaptive_thresh = cv2.adaptiveThreshold(
    blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, -5
)

In [ ]:
plt.figure(figsize=(8, 10))
plt.imshow(adaptive_thresh, cmap="gray")
plt.axis("off")
plt.title("Adaptive Threshold")
plt.show()

In [ ]:
# Find contours
contours, hierarchy = cv2.findContours(
    adaptive_thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE
)

# Darken the image (it is supported even though it shows a type error)
darkened_image = (image_rgb * 0.6).astype(np.uint8)  # type: ignore

# Draw contours on darkened image
contour_image = darkened_image.copy()
_ = cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

In [ ]:
# Plot the result
plt.figure(figsize=(8, 10))
plt.imshow(contour_image)
plt.axis("off")
plt.title(f"Contours on Darkened Image (Total: {len(contours)} contours)")
plt.show()

In [ ]:
# Filter contours to find circles with diameter 9-15 and white/gray fill (no color)
filtered_contours = []
min_diameter = 8
max_diameter = 18
min_radius = min_diameter / 2
max_radius = max_diameter / 2

# Thresholds
max_std_dev_threshold = 20
# low saturation means gray/white (no color)
max_saturation = 80  # Easy to change: lower = more strict (0-255 scale)

# Convert to HSV for saturation check
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

# For visualization
i = 0
fig = plt.figure(figsize=(17, 6))
ncols, nrows = 20, 4
subfigs = fig.subfigures(nrows, ncols)
j = 0


def render_contour_subfigures(
    filtered_contours,
    subfigs,
    contour,
    x,
    y,
    radius,
    mean_intensity,
    std_intensity,
    mean_saturation,
    meets_filter_criteria,
    meets_intensity_threshold,
    is_variance_acceptable,
):
    global i, j
    i += 1

    if i % 1 == 0 and j < ncols * nrows:
        # Get bounding box
        x_int, y_int = int(x), int(y)
        r_int = int(radius)

        # Define crop region with padding
        padding = 10
        y1 = max(0, y_int - r_int - padding)
        y2 = min(image_rgb.shape[0], y_int + r_int + padding)
        x1 = max(0, x_int - r_int - padding)
        x2 = min(image_rgb.shape[1], x_int + r_int + padding)

        # Crop the region
        cropped_img = image_rgb[y1:y2, x1:x2].copy()

        # Dim the image
        dimmed = (cropped_img * 0.9).astype(np.uint8)

        # Adjust contour coordinates for cropped image
        contour_shifted = contour - [x1, y1]

        # Draw contour on dimmed image
        color = (0, 255, 0, 150) if meets_filter_criteria else (255, 0, 0, 150)
        cv2.drawContours(dimmed, [contour_shifted], -1, color, 1)

        # Create subplot
        subfig = subfigs[j // ncols, j % ncols]
        ax = subfig.add_subplot(1, 1, 1)
        ax.imshow(dimmed)
        ax.axis("off")
        ax.set_title(
            (
                f"#{i} / {len(filtered_contours)}\n"
                f"μ={mean_intensity:.0f}, σ={std_intensity:.0f}\n"
                f"sat={mean_saturation:.0f}\n"
                f"meets={meets_filter_criteria}\n"
                f"intensity={meets_intensity_threshold}\n"
                f"variance={is_variance_acceptable}\n"
                f"saturation={mean_saturation:.0f} {mean_saturation < max_saturation}"
            ),
            fontsize=6,
        )
        j += 1


for contour in contours:
    # Calculate contour area and perimeter
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)

    # Get minimum enclosing circle
    (x, y), radius = cv2.minEnclosingCircle(contour)

    # Calculate circularity (1.0 is perfect circle)
    if perimeter == 0:
        continue

    circularity = 4 * np.pi * area / (perimeter * perimeter)

    # print(f"Contour at ({x:.1f}, {y:.1f}) with radius {radius:.1f} and circularity {circularity:.2f}, area {area:.1f}, perimeter {perimeter:.1f}")

    # if not( 50 <= y <= 1300 and 2400 <= x <= 3400):
    #     continue

    # Skip very small contours
    if area < 30 or perimeter <= 10:
        continue

    # Check if it's circular enough
    if circularity <= 0.4:
        continue

    # Check if diameter is in range
    if not (min_radius <= radius <= max_radius):
        continue

    # Create mask for the circle region
    mask = np.zeros(gray.shape, dtype=np.uint8)
    cv2.circle(mask, (int(x), int(y)), int(radius) - 2, 255, -1)

    # Extract the region from original grayscale image
    pixels = gray[mask == 255]

    # Check if region is relatively white/gray (high intensity)
    mean_intensity = pixels.mean()
    std_intensity = pixels.std()

    # Extract HSV region to check saturation (color)
    hsv_masked = cv2.bitwise_and(image_hsv, image_hsv, mask=mask)
    saturation_values = hsv_masked[:, :, 1][mask == 255]
    mean_saturation = np.mean(saturation_values)

    meets_intensity_threshold = mean_intensity > 100
    is_variance_acceptable = std_intensity < max_std_dev_threshold
    saturation_meets_threshold = mean_saturation < max_saturation
    meets_filter_criteria = (
        meets_intensity_threshold
        and is_variance_acceptable
        and saturation_meets_threshold
    )
    render_contour_subfigures(
        filtered_contours,
        subfigs,
        contour,
        x,
        y,
        radius,
        mean_intensity,
        std_intensity,
        mean_saturation,
        meets_filter_criteria,
        meets_intensity_threshold,
        is_variance_acceptable,
    )

    # Filter: high intensity, low std deviation, AND low saturation (no color)
    if meets_filter_criteria:
        filtered_contours.append(contour)

plt.tight_layout()
plt.show()

# Visualize all filtered contours on full image
filtered_image = darkened_image.copy()
cv2.drawContours(filtered_image, filtered_contours, -1, (0, 255, 0), 2)

plt.figure(figsize=(12, 8))
plt.imshow(filtered_image)
plt.axis("off")
plt.title(
    f"Filtered Circular Contours (White/Gray, No Color) (Total: {len(filtered_contours)} circles)"
)
plt.show()

print(f"Found {len(filtered_contours)} circles matching criteria")

In [ ]:
plt.tight_layout()
plt.show()

# Visualize all filtered contours on full image
filtered_image = darkened_image.copy()
cv2.drawContours(filtered_image, filtered_contours, -1, (0, 255, 0), 2)

plt.figure(figsize=(12, 8))
plt.imshow(filtered_image)
plt.axis("off")
plt.title(
    f"Filtered Circular Contours (White/Gray, No Color) (Total: {len(filtered_contours)} circles)"
)
plt.show()

print(f"Found {len(filtered_contours)} circles matching criteria")

In [ ]:
hsv_colors: list[tuple[int, int, int]] = []

for contour in filtered_contours:
    (x, y), radius = cv2.minEnclosingCircle(contour)
    circle_inner = cv2.circle(
        np.zeros_like(gray), (int(x), int(y)), int(radius) + 2, 255, -1
    )
    circle_outer = cv2.circle(
        np.zeros_like(gray), (int(x), int(y)), int(radius) + 6, 255, -1
    )
    donut = circle_outer - circle_inner
    scalar = cv2.mean(image_rgb, mask=donut)
    is_tuple_of_floats = isinstance(scalar, tuple) and all(
        isinstance(x, float) for x in scalar
    )
    if not is_tuple_of_floats:
        raise ValueError(
            f"Expected scalar to be a tuple of floats, got {type(scalar)} with values {scalar}"
        )
    r, g, b, _ = scalar  # type: ignore
    h, s, v = cv2.cvtColor(
        np.uint8([[[int(r), int(g), int(b)]]]), cv2.COLOR_RGB2HSV # type: ignore
    )[0][0]
    
    mean_hsv = (int(h), int(s), int(v))
    hsv_colors.append(mean_hsv)

In [ ]:
# Create a copy of the original image to show the donuts
donut_visualization = image_rgb.copy()

for contour in filtered_contours:
    (x, y), radius = cv2.minEnclosingCircle(contour)

    # Create the donut mask
    circle_inner = cv2.circle(
        np.zeros_like(gray), (int(x), int(y)), int(radius) + 2, 255, -1
    )
    circle_outer = cv2.circle(
        np.zeros_like(gray), (int(x), int(y)), int(radius) + 6, 255, -1
    )
    donut = circle_outer - circle_inner

    # Invert the colors in the donut region
    donut_mask_3ch = cv2.cvtColor(donut, cv2.COLOR_GRAY2RGB)
    donut_region = donut_mask_3ch > 0
    donut_visualization[donut_region] = 255 - donut_visualization[donut_region]

plt.figure(figsize=(8, 10))
plt.imshow(donut_visualization)
plt.axis("off")
plt.title(f"Donut Regions with Inverted Colors ({len(filtered_contours)} circles)")
plt.show()

In [ ]:
len(hsv_colors)

In [ ]:
# Define reference colors
reference_colors_rgb: dict[str, tuple[int, int, int]] = {
    "Red": (183, 74, 65),
    "Yellow": (206, 147, 78),
    "Green": (118, 136, 94),
    "Blue": (93, 124, 169),
    "Pink": (193, 87, 121),
}

reference_colors: dict[str, tuple[int, int, int]] = {
    item[0]: cv2.cvtColor(np.uint8([[item[1]]]), cv2.COLOR_RGB2HSV)[0][0]  # type: ignore
    for item in reference_colors_rgb.items()
}

reference_colors

In [ ]:
# Convert HSV colors to Cartesian coordinates for 3D plotting
# HSV is in cylindrical coordinates: H (hue) is angle, S (saturation) is radius, V (value) is height

import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import clear_output, display

from tacta import hsv_to_cartesian

# Create output widget for displaying clicked point images
output_widget = widgets.Output()


def display_cropped_circle(contour_index):
    """Display a cropped region around the clicked circle"""
    contour = filtered_contours[contour_index]
    (x, y), radius = cv2.minEnclosingCircle(contour)

    x_int, y_int = int(x), int(y)
    r_int = int(radius)

    # Define crop region with padding
    padding = 20
    y1 = max(0, y_int - r_int - padding)
    y2 = min(image_rgb.shape[0], y_int + r_int + padding)
    x1 = max(0, x_int - r_int - padding)
    x2 = min(image_rgb.shape[1], x_int + r_int + padding)

    # Crop the region
    cropped_img = image_rgb[y1:y2, x1:x2].copy()

    # Dim the image
    dimmed = (cropped_img * 0.9).astype(np.uint8)

    # Adjust contour coordinates for cropped image
    contour_shifted = contour - [x1, y1]

    # Draw contour on dimmed image
    cv2.drawContours(dimmed, [contour_shifted], -1, (0, 255, 0), 1)

    # Display in output widget
    with output_widget:
        clear_output(wait=True)
        fig_crop, ax = plt.subplots(figsize=(4, 4))
        ax.imshow(dimmed)
        ax.axis("off")
        ax.set_title(
            f"Circle {contour_index + 1} at ({x_int}, {y_int})\nHSV: {hsv_colors[contour_index]}"
        )
        plt.tight_layout()
        plt.show()


# Prepare data for plotting
x_coords = []
y_coords = []
z_coords = []
colors_rgb = []

for hsv in hsv_colors:
    h, s, v = hsv[0], hsv[1], hsv[2]

    # Convert HSV to cylindrical coordinates
    angle, radius, height = hsv_to_cartesian(h, s, v)

    # Convert to Cartesian coordinates
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z = height

    # Convert HSV to RGB for point color
    hsv_normalized = np.uint8([[[h, s, v]]])  # type: ignore
    rgb = cv2.cvtColor(hsv_normalized, cv2.COLOR_HSV2RGB)[0][0]  # type: ignore
    color_rgb = f"rgb({rgb[0]}, {rgb[1]}, {rgb[2]})"

    x_coords.append(x)
    y_coords.append(y)
    z_coords.append(z)
    colors_rgb.append(color_rgb)

# Prepare reference colors for plotting
ref_x_coords = []
ref_y_coords = []
ref_z_coords = []
ref_colors_rgb = []
ref_names = []

for color_name, ref_hsv in reference_colors.items():
    h, s, v = ref_hsv

    # Convert to Cartesian coordinates
    angle, radius, height = hsv_to_cartesian(h, s, v)
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z = height

    # Convert HSV to RGB for point color
    hsv_normalized = np.uint8([[[h, s, v]]])  # type: ignore
    rgb = cv2.cvtColor(hsv_normalized, cv2.COLOR_HSV2RGB)[0][0]  # type: ignore
    color_rgb = f"rgb({rgb[0]}, {rgb[1]}, {rgb[2]})"

    ref_x_coords.append(x)
    ref_y_coords.append(y)
    ref_z_coords.append(z)
    ref_colors_rgb.append(color_rgb)
    ref_names.append(color_name)

# Create 3D scatter plot for data points
scatter = go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode="markers",
    marker=dict(size=5, color=colors_rgb, opacity=1),
    name="Detected colors",
)

# Create 3D scatter plot for reference colors
reference_scatter = go.Scatter3d(
    x=ref_x_coords,
    y=ref_y_coords,
    z=ref_z_coords,
    mode="markers+text",
    marker=dict(size=6, color=ref_colors_rgb, symbol="diamond"),
    text=ref_names,
    textposition="top center",
    name="Reference colors",
)

# Create FigureWidget with both traces for interactivity
fig = go.FigureWidget([scatter, reference_scatter])


# Add click handler - attach to scatter trace only
def on_click(trace, points, selector):
    if points.point_inds:
        point_index = points.point_inds[0]
        display_cropped_circle(point_index)


# Attach handler to the first trace (data points)
fig.data[0].on_click(on_click)

# Update layout
fig.update_layout(
    title=f"HSV Color Distribution in 3D Space ({len(hsv_colors)} colors) - Click to view",
    scene=dict(
        xaxis_title="X (Saturation * cos(Hue))",
        yaxis_title="Y (Saturation * sin(Hue))",
        zaxis_title="Z (Value)",
    ),
)

display(fig)
display(output_widget)

In [ ]:
from sklearn.cluster import KMeans

# Create output widget for displaying clicked point images
output_widget_kmeans = widgets.Output()


def display_cropped_circle_kmeans(contour_index):
    """Display a cropped region around the clicked circle with cluster info"""
    contour = filtered_contours[contour_index]
    (x, y), radius = cv2.minEnclosingCircle(contour)

    x_int, y_int = int(x), int(y)
    r_int = int(radius)

    # Define crop region with padding
    padding = 20
    y1 = max(0, y_int - r_int - padding)
    y2 = min(image_rgb.shape[0], y_int + r_int + padding)
    x1 = max(0, x_int - r_int - padding)
    x2 = min(image_rgb.shape[1], x_int + r_int + padding)

    # Crop the region
    cropped_img = image_rgb[y1:y2, x1:x2].copy()

    # Adjust contour coordinates for cropped image
    contour_shifted = contour - [x1, y1]

    # Draw contour on cropped image
    cluster_id = cluster_labels[contour_index]
    cv2.drawContours(cropped_img, [contour_shifted], -1, (0, 255, 0), 2)

    # Display in output widget
    with output_widget_kmeans:
        clear_output(wait=True)
        fig_crop, ax = plt.subplots(figsize=(4, 4))
        ax.imshow(cropped_img)
        ax.axis("off")
        ax.set_title(
            f"Circle {contour_index + 1} at ({x_int}, {y_int})\n"
            f"Cluster: {cluster_id}\n"
            f"HSV: {hsv_colors[contour_index]}"
        )
        plt.tight_layout()
        plt.show()


# Prepare data for k-means clustering
# Convert HSV colors to numpy array
hsv_array = np.array(hsv_colors)[:, :3]  # Extract H, S, V values

# Perform k-means clustering
k = 7
kmeans = KMeans(n_clusters=k, random_state=1, n_init=10)
cluster_labels = kmeans.fit_predict(hsv_array)
cluster_centers = kmeans.cluster_centers_

# Prepare data for plotting
x_coords = []
y_coords = []
z_coords = []
colors_rgb = []

# Convert cluster centers from HSV to RGB for coloring
cluster_colors = []
for center in cluster_centers:
    h, s, v = center
    hsv_normalized = np.uint8([[[h, s, v]]])  # type: ignore
    rgb = cv2.cvtColor(hsv_normalized, cv2.COLOR_HSV2RGB)[0][0]  # type: ignore
    cluster_colors.append(f"rgb({rgb[0]}, {rgb[1]}, {rgb[2]})")

for i, hsv in enumerate(hsv_colors):
    h, s, v = hsv[0], hsv[1], hsv[2]

    # Convert to Cartesian coordinates
    angle, radius, height = hsv_to_cartesian(h, s, v)

    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z = height

    # Color by cluster center's actual color
    cluster_id = cluster_labels[i]
    color = cluster_colors[cluster_id]

    x_coords.append(x)
    y_coords.append(y)
    z_coords.append(z)
    colors_rgb.append(color)

# Create scatter plot for data points
scatter = go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode="markers",
    marker=dict(size=5, color=colors_rgb, opacity=0.8),
    name="Data points",
)

# Prepare cluster center coordinates
center_x = []
center_y = []
center_z = []
center_colors = []

for i, center in enumerate(cluster_centers):
    h, s, v = center
    angle, radius, height = hsv_to_cartesian(h, s, v)

    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z = height

    center_x.append(x)
    center_y.append(y)
    center_z.append(z)
    center_colors.append("black")

# Create scatter plot for cluster centers
centers = go.Scatter3d(
    x=center_x,
    y=center_y,
    z=center_z,
    mode="markers",
    marker=dict(size=6, color="black", symbol="x", line=dict(color="white", width=2)),
    name="Cluster centers",
)

# Create FigureWidget with both traces for interactivity
fig = go.FigureWidget([scatter, centers])


# Add click handler for data points - attach to scatter trace only
def on_click_kmeans(trace, points, selector):
    if points.point_inds:
        point_index = points.point_inds[0]
        display_cropped_circle_kmeans(point_index)


# Attach handler to the first trace (data points)
fig.data[0].on_click(on_click_kmeans)

# Update layout
fig.update_layout(
    title=f"K-Means Clustering (k={k}) of HSV Colors ({len(hsv_colors)} points) - Click to view",
    scene=dict(
        xaxis_title="X (Saturation * cos(Hue))",
        yaxis_title="Y (Saturation * sin(Hue))",
        zaxis_title="Z (Value)",
    ),
)

display(fig)
display(output_widget_kmeans)

# Print cluster information
print("\nCluster distribution:")
for i in range(k):
    count = np.sum(cluster_labels == i)
    print(f"Cluster {i}: {count} points")

In [ ]:
# Interpret cluster colors based on HSV centers
def hsv_to_color_name_distance(h, s, v):
    """Convert HSV values to color name based on closest reference color"""
    # Current point in HSV
    current = np.array([h, s, v])

    # Find closest reference color
    min_distance = float("inf")
    closest_color = "Unknown"

    for color_name, ref_hsv in reference_colors.items():
        ref = np.array(ref_hsv)
        # Euclidean distance in HSV space
        distance = np.linalg.norm(current - ref)

        if distance < min_distance:
            min_distance = distance
            closest_color = color_name

    return closest_color


# Assign each cluster to its closest color (one-to-one mapping)
cluster_to_color = {}
used_colors = set()

print("\nCluster color analysis:")
print("-" * 60)

# Create a list of (cluster_id, color_name, distance) for all combinations
assignments = []
for i, center in enumerate(cluster_centers):
    h, s, v = center
    current = np.array([h, s, v])

    for color_name, ref_hsv in reference_colors.items():
        ref = np.array(ref_hsv)
        distance = np.linalg.norm(current - ref)
        assignments.append((i, color_name, distance))

# Sort by distance and assign greedily (closest matches first)
assignments.sort(key=lambda x: x[2])

for cluster_id, color_name, distance in assignments:
    if cluster_id not in cluster_to_color and color_name not in used_colors:
        cluster_to_color[cluster_id] = color_name
        used_colors.add(color_name)

# Print results
for i, center in enumerate(cluster_centers):
    h, s, v = center
    color_name = cluster_to_color.get(i, "Unknown")
    count = np.sum(cluster_labels == i)
    print(f"Cluster {i}: {color_name}")
    print(f"  HSV: H={h:.1f}°, S={s:.1f}, V={v:.1f}")
    print(f"  Count: {count} circles")
    print()